In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
import requests
import time
from random import uniform
import pandas as pd

In [3]:
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)
#여성
driver.get('https://wusinsa.musinsa.com/app/items/lists/001/?category=&d_cat_cd=001&u_cat_cd=&brand=&sort=pop&sub_sort=&display_cnt=90&page=1&page_kind=category&list_kind=small&free_dlv=&ex_soldout=&sale_goods=&exclusive_yn=&price=&color=&a_cat_cd=&sex=28&size=&tag=&popup=&brand_favorite_yn=&goods_favorite_yn=&blf_yn=&campaign_yn=&price1=&price2=&brand_favorite=&goods_favorite=&chk_exclusive=&chk_sale=&chk_soldout=')

In [4]:
link_list = []

In [5]:
try:
    for src in range(3):
        for page in range(4,14):

            ## 'div.listLs_L.buyerL > div > h2 > a' 에 해당하는 elements를 모두 받아온다
            li = driver.find_elements_by_css_selector('div.boxed-list-wrapper > div.sorter-box.box > div > div > a')

            #페이지에 있는 상품 url 저장
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            links = soup.select('div.li_inner > div.article_info > p.list_info > a')

            for link in links:
                link_list.append('https://store.musinsa.com'+link['href'])

            ## 다음 페이지 클릭
            driver.find_element_by_xpath('//*[@id="contentsItem_list"]/div[2]/div[1]/div/div/a[%s]' %page).click()
            time.sleep(0.5)
except:
    print("더 이상 페이지가 없습니다.")

더 이상 페이지가 없습니다.


In [6]:
link_list = list(set(link_list))

In [7]:
len(link_list)

1788

In [9]:
# csv 파일로 저장
import pandas as pd
results = pd.DataFrame(link_list, columns=['link_addr'])
results.to_csv('./link_musinsa_W.csv', header=False) #, mode='a', header=False)

In [10]:
# csv 파일 가져오기
import csv
f = open('./link_musinsa_W.csv','rt')
link_addr_file = csv.reader(f)

In [11]:
link_list = []
for row in link_addr_file :
    link_list.append(row[1])

In [12]:
link_list

['https://store.musinsa.com/app/product/detail/1360803/0',
 'https://store.musinsa.com/app/product/detail/1342833/0',
 'https://store.musinsa.com/app/product/detail/1358729/0',
 'https://store.musinsa.com/app/product/detail/1354050/0',
 'https://store.musinsa.com/app/product/detail/1363721/0',
 'https://store.musinsa.com/app/product/detail/1234641/0',
 'https://store.musinsa.com/app/product/detail/1275024/0',
 'https://store.musinsa.com/app/product/detail/1359951/0',
 'https://store.musinsa.com/app/product/detail/953273/0',
 'https://store.musinsa.com/app/product/detail/1352130/0',
 'https://store.musinsa.com/app/product/detail/1353069/0',
 'https://store.musinsa.com/app/product/detail/1362775/0',
 'https://store.musinsa.com/app/product/detail/1275015/0',
 'https://store.musinsa.com/app/product/detail/1357458/0',
 'https://store.musinsa.com/app/product/detail/1354730/0',
 'https://store.musinsa.com/app/product/detail/1363391/0',
 'https://store.musinsa.com/app/product/detail/1275005/0'

In [18]:
len(link_list)

1788

In [19]:
#상세 페이지 들어가서 원하는 정보 저장
i = 0

while i < len(link_list):
    text_list = []
    
    driver.get(link_list[i])
    print(link_list[i])
    
    f = uniform(1.3, 1.6)
    time.sleep(f)
    
    a = ""
    a = link_list[i].split("/detail/")
    a[1][:6]
    
    i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        images = soup.select('div.right_area.page_detail_product > div.right_contents.product_info_contents > img')
        name = soup.find('h3','_2IA5sp7BRM').get_text()
        price = soup.find('span','_1dpDPg-OQb').get_text()
        tag=soup.find('div','_24FKuaBdoK').get_text()
        info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]
        detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)]
        texts = soup.find_all('p', 'se_textarea')


        if not os.path.isdir(a[1][:6]):
            os.mkdir(a[1][:6])     

        for text in texts:
            text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
            text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = ({'category':'여성/상의', 'name': [name], 'price': [price],'tag':[tag], 'size':[text_list2]})
        df1 = pd.DataFrame(data_1)

        data_2 = [detail]
        df2 = pd.DataFrame(data_2, columns = info)

        df3 = pd.concat([df1,df2], axis=1)

        df3.to_csv(".\\"+a[1][:10]+"\\csv_"+a[1][:10]+".csv", index = True, encoding='utf-8')

        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://wusinsa.musinsa.com//"})
            src_image = src_page.content
            with open(".\\"+a[1][:10]+"\\img_"+a[1][:10]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)

    except:
        print("상품정보 가져오기 오류발생")
        pass

https://store.musinsa.com/app/product/detail/1360803/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342833/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1358729/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354050/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1363721/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1234641/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1275024/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1359951/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/953273/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1352130/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353069/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362775/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1275015/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1357458/0
상품정보 가져오기 오류발생
https://store.musinsa

상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354511/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342445/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1343136/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1357679/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354800/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1344358/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360269/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1269852/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1365265/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1340338/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1346764/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1348813/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1346058/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1338467/0
상품정보 가져오기 오류발생
https

https://store.musinsa.com/app/product/detail/1337212/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354182/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355639/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354837/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1359956/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341698/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1361167/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1312754/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1339015/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1358112/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1361189/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362304/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1336559/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1364352/0
상품정보 가져오기 오류발생
https://store.musins

상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1118437/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1339290/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1339557/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353282/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1340589/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342724/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362130/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342696/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355905/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1337341/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1337914/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1351022/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353331/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1349755/0
상품정보 가져오기 오류발생
https

https://store.musinsa.com/app/product/detail/1345457/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1356634/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354705/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341661/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354700/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353207/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355697/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1351367/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1339871/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360057/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1350395/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1363712/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355535/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360304/0
상품정보 가져오기 오류발생
https://store.musins

...

https://store.musinsa.com/app/product/detail/1336548/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1142324/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360306/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341745/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360026/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1356636/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360941/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355169/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362499/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342731/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1359159/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341696/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341687/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1337157/0
상품정보 가져오기 오류발생
https://store.musins

상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1275010/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362680/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362408/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1351634/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1352689/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342539/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/836977/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1338951/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1352915/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/960382/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1342146/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360161/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1343462/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1341670/0
상품정보 가져오기 오류발생
https:/

https://store.musinsa.com/app/product/detail/1310958/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1351869/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353371/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1355660/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362710/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354137/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1362682/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1350200/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353372/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353139/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1354241/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1275020/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/713074/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1361608/0
상품정보 가져오기 오류발생
https://store.musinsa

상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1344058/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1361946/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1359641/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1364182/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1346057/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/932129/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1353228/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1336557/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/994588/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1360293/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1363691/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1118693/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1359651/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1345521/0
상품정보 가져오기 오류발생
https:/

KeyboardInterrupt: 